In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.1 MB/s eta 0:00:00


In [2]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.6 MB/s eta 0:00:00


In [3]:
import torch
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import urllib.request
import re

In [4]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [5]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [6]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  
        return len(self._data)

    def __getitem__(self, idx):  
        turn = self._data.iloc[idx]
        q = turn["Q"]  
        q = re.sub(r"([?.!,])", r" ", q)  

        a = turn["A"]  
        a = re.sub(r"([?.!,])", r" ", a)  

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

    
        if q_len > self.max_len:
            a_len = self.max_len - q_len        
            if a_len <= 0:       
                q_toked = q_toked[-(int(self.max_len / 2)) :]   
                q_len = len(q_toked)
                a_len = self.max_len - q_len             
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)


        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len       
            if a_len <= 0:       
                q_toked = q_toked[-(int(self.max_len / 2)) :]   
                q_len = len(q_toked)
                a_len = self.max_len - q_len              
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]


        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
 
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
  
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]
   
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
  
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]


        return (token_ids, np.array(mask), labels_ids)

In [7]:
def train_data():
      urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",filename="ChatBotData.csv",)
      train_dt = pd.read_csv('ChatBotData.csv')      
      return train_dt

In [8]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [10]:
Chatbot_Data = train_data()
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_batch,)

In [11]:
print(len(Chatbot_Data))

11823


In [12]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
torch.save(model.state_dict(),'./output_model_best')

In [13]:
learning_rate = 3e-5 
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps = 1e-8)

epochs = 3 
Sneg = -1e18

total_steps = len(train_dataloader) * epochs
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

In [14]:
from tqdm.notebook import tqdm

In [15]:
def train_runner():    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.to(device)
    model.train()
    torch.set_grad_enabled(True)

    global_total_step = len(train_dataloader) * epochs

    with tqdm(total = global_total_step, unit = 'step') as t:

        total = 0
        total_loss = 0

        for epoch in range(epochs):
            print(f"epoch_num : {epoch+1}")
            for batch_idx, train_base in enumerate(train_dataloader):

                optimizer.zero_grad()
                token_ids, mask, label = train_base

                token_ids = token_ids.to(device)
                mask = mask.to(device)
                label = label.to(device)

                out = model(token_ids)
                out = out.logits     
                
                mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
                mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

                loss = criterion(mask_out.transpose(2, 1), label)
                avg_loss = loss.sum() / mask.sum()
                avg_loss.backward()
                    
                optimizer.step()
                
                global_total_step += 1

                batch_loss = avg_loss.item()
                total += 1
                total_loss += batch_loss

                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)

        scheduler.step()


In [ ]:
model = model.to('cuda')

In [16]:
train_runner()

  0%|          | 0/1110 [00:00<?, ?step/s]

epoch_num : 1


<ipython-input-8-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


epoch_num : 2
epoch_num : 3


학습 진행 ~~(전체데이터 X)\
CPU 기반 학습 -> CUDA 환경으로 학습 시킬때 변경해야할 변수들 여러개 고려해야했음.\
(+ longTensor // 위에 토큰화 할때 np.array 사용된거 tensor형태로 바로 받을수 있게 변경해야할듯)\
hyperparameter - finetune ->> wandb 시각화\
\
q a 받는 과정 포함된 class도 수정필요\
model -> huggingface\
\
kakao - KOGPT 모델도 실습 예정



In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [22]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))      

user > 안녕
Chatbot > 안녕하세요
user > 이름이 뭐야?
Chatbot > 저는 위로해드리는 위로봇입니다
user > 벌써 저녁이네
Chatbot > 맛있는 거 드세요
user > 졸리다
Chatbot > 낮잠을 주무세요
user > 내일 일찍일어나야해
Chatbot > 오늘보다 조금 더 잘 살 수 있을 거예요
user > 세종대 알아?
Chatbot > 저도 궁금하네요
user > 점심 메뉴 추천좀
Chatbot > 맛있는 거 드세요
user > quit
